In [1]:
!date

Thu Apr  6 14:13:37 UTC 2023


In [2]:
!pip install tcc-cft-tool-0.3.4-py_0.tar.bz2

Processing ./tcc-cft-tool-0.3.4-py_0.tar.bz2
ERROR: file:///mnt/tcc-cft-tool-0.3.4-py_0.tar.bz2 does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


## Demostration of using `tcc_cft_tool` package to run CFT API calculation
Author: Yafan Tang


In [2]:
import pandas as pd

from tcc_cft_tool import CftTool, CftTemplate,CftExcel
from tcc_s3 import SSM, S3

In [3]:
# retrieve app key from s3
# skip this cell, if you have had the key.
ssm = SSM("production")
app_key = ssm.get_parameter("production.team-science.cft-api.app-key")

/Users/yafan.tang/opt/anaconda3/envs/cft_v0.3.2/lib/python3.9/site-packages/tcc_s3/aws_service.py:26: FutureWarning: Passing an environment argument is deprecated. In the future this argument will be ignored.
  warnings.warn(


In [4]:
# get json example data
template = CftTemplate()
request_ls = template.input_json_template

In [5]:
# get 
# prepare a example data yourself using latest excel template
input_path =  "/Users/yafan.tang/Library/CloudStorage/OneDrive-Bayer/CFT-DFS-EMEA/Request/SUS0135/EMEA_PilotFarms_Datacollection_2022.xlsx"
excel_io = pd.ExcelFile(input_path, engine="openpyxl")
cft_excel = CftExcel(excel_io)
request_ls = cft_excel.transform_excel_to_request()[:5]

Transforming sheet: 0-2 Farm-Crop-Soil...
Transforming sheet: 3.1 Fertilizer...


/Users/yafan.tang/opt/anaconda3/envs/cft_v0.3.2/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/yafan.tang/opt/anaconda3/envs/cft_v0.3.2/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


Transforming sheet: 3.2 Crop_Protection...


/Users/yafan.tang/opt/anaconda3/envs/cft_v0.3.2/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


Transforming sheet: 4.1 Direct_Energy...
Transforming sheet: 4.2 Operation_Energy...


/Users/yafan.tang/opt/anaconda3/envs/cft_v0.3.2/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/yafan.tang/opt/anaconda3/envs/cft_v0.3.2/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


Transforming sheet: 5 Irrigation...
Transforming sheet: 6 Carbon...
Transforming sheet: 7 Transport...


/Users/yafan.tang/opt/anaconda3/envs/cft_v0.3.2/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/yafan.tang/opt/anaconda3/envs/cft_v0.3.2/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/yafan.tang/opt/anaconda3/envs/cft_v0.3.2/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/yafan.tang/opt/anaconda3/envs/cft_v0.3.2/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


7 Transport is empty
Transforming sheet: 8.1 Potato (Processing)...
8.1 Potato (Processing) is empty
Transforming sheet: 8.2 Potato (Storage)...
8.2 Potato (Storage) is empty


/Users/yafan.tang/opt/anaconda3/envs/cft_v0.3.2/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/yafan.tang/opt/anaconda3/envs/cft_v0.3.2/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [6]:
cft = CftTool(request_ls, app_key,cft_major_version=1)
cft.validate_request()
validation_df = cft.validation_result
assert (~validation_df["validate_status"]).sum() ==0 

Validating schema of CFT API requests...


In [7]:
# check if any case failed data validation
validation_df[~validation_df["validate_status"]]

,AssessmentID,request,validate_status,validate_error


In [8]:
# interact with CFT API
cft.call_api()

Calling CFT API ...
Calculation result acquired!


/Users/yafan.tang/opt/anaconda3/envs/cft_v0.3.2/lib/python3.9/site-packages/tcc_cft_tool/cft_tool.py:133: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  api_df.groupby(["validate_status", "group"])


In [9]:
# show detail results
cft.api_result

,AssessmentID,request,validate_status,validate_error,return
0,A59::2021,{'farm': {'average_temperature': {'value': 11....,True,None,"{'farm': {'farm_identifier': 'A59'}, 'summary'..."
1,AAN DE WATERTOREN::2022,{'farm': {'average_temperature': {'value': 11....,True,None,{'farm': {'farm_identifier': 'AAN DE WATERTORE...
2,AKAZIENBUSCH::2022,{'farm': {'average_temperature': {'value': 11....,True,None,"{'farm': {'farm_identifier': 'AKAZIENBUSCH'}, ..."
3,ARCO LANGRE::2022,{'farm': {'average_temperature': {'value': 18....,True,None,"{'farm': {'farm_identifier': 'ARCO LANGRE'}, '..."
4,AUFELD::2021,{'farm': {'average_temperature': {'value': 11....,True,None,"{'farm': {'farm_identifier': 'AUFELD'}, 'summa..."


In [10]:
# check if any returns contain error
cft.api_result[cft.api_result["return"].astype(str).str.contains("error")]

,AssessmentID,request,validate_status,validate_error,return


In [11]:
# parse summary section
cft.parse_summary()

,AssessmentID,result,value,unit
0,A59::2021,emissions_total,32191.05,kgCO2e
1,A59::2021,emissions_per_area,2160.47,kgCO2e/ha
2,A59::2021,emissions_per_product,0.04,kgCO2e/kg
3,A59::2021,soil_organic_carbon,0.81,%
4,A59::2021,machinery_fuel_usage,0.00,litre
0,AAN DE WATERTOREN::2022,emissions_total,2308.03,kgCO2e
1,AAN DE WATERTOREN::2022,emissions_per_area,384.67,kgCO2e/ha
2,AAN DE WATERTOREN::2022,emissions_per_product,0.40,kgCO2e/kg
3,AAN DE WATERTOREN::2022,soil_organic_carbon,0.52,%
4,AAN DE WATERTOREN::2022,machinery_fuel_usage,0.00,litre


In [12]:
# parse detail section
cft.parse_detail()

,AssessmentID,source,CO2,N2O,CH4,total_CO2e,total_CO2e_per_area,total_CO2e_per_product
0,A59::2021,fertiliser_production,32599.30,0.00,0.0,32599.30,2187.87,0.04
1,A59::2021,fertiliser_soil,0.00,36.28,0.0,9904.84,664.75,0.01
2,A59::2021,field_energy,4472.92,0.00,0.0,4472.92,300.20,0.01
3,A59::2021,irrigation,0.00,0.00,0.0,0.00,0.00,0.00
4,A59::2021,machinery,0.00,0.00,0.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...
8,AUFELD::2021,residue,0.00,6.94,0.0,1895.89,296.70,0.05
9,AUFELD::2021,seed,0.00,0.00,0.0,0.00,0.00,0.00
10,AUFELD::2021,storage,0.00,0.00,0.0,0.00,0.00,0.00
11,AUFELD::2021,transportation,0.00,0.00,0.0,0.00,0.00,0.00
